In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Parameters

In [3]:

torch.manual_seed(1337)
batch_size = 32 # How many independent sequences we will process in parallel
block_size = 8 # The maximum "context" length for predictions

Read the data from a file (We can choose a new file below)

In [4]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print("length of dataset (chars)", len(text))

print("First 1000 characters:")
print(text[:1000])

length of dataset (chars) 1115394
First 1000 characters:
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
spe

We find all the unique characters in the dataset (Our vocabulary so to speak)

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


Very basic mapping and encoding (from char to int)

In [6]:
# Map characters to integers (and back)
stringToInteger = { char:integer for integer, char in enumerate(chars) }
integerToString = { integer:char for integer, char in enumerate(chars) }

# Encode: take a string, output a list of integers
encode = lambda string: [stringToInteger[char] for char in string]

# Decode: take a list of integers, output a string
decode = lambda intList: ''.join([integerToString[integer] for integer in intList]) 

print(encode("Good day sir!"))
print(decode((encode("Good day sir!"))))

[19, 53, 53, 42, 1, 42, 39, 63, 1, 57, 47, 56, 2]
Good day sir!


Encode and store our data in a pytorch tensor.

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # This is what the model will see from the first 1000 characters

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

Split into training and testing data

In [8]:
n = int(0.9*len(data)) # Use first 90% for taining and use the last 10% for testing
train_data = data[:n]
test_data = data[n:]

Chunking

In [13]:
def get_batch(split):
    # generate a small batch of inputs x and targets y
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
# print('inputs:')
# print(xb.shape)
# print(xb)
# print('targets:')
# print(yb.shape)
# print(yb)
# print('-----')

# for b in range(batch_size): # batch dimension
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target is {target}")

In [14]:
# (We work with this existing class)
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            # Reshape logits so we can calculate loss
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # loss function
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get prediction
            logits, loss = self(idx)
            # focus on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [15]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor(4.7744, grad_fn=<NllLossBackward0>)

.rKpUuAABpf,BaO?YuCBpTeJJR:o&nkQJHfISyJrfy$PWi,q?!BKYKtJgrcfwSH
auiR;'HebkgEMPnNx;B;tjiRkppRLpiSABKG


Train the model

In [16]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [17]:
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4537193775177


In [18]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))


Whaig t lding fin y h herof, ou br ald sa youm ble r l s.
TEO, th,
OLAMINouy w hel challouthilerrs Pror w?
Frmes as h'dr,
NI NTwetatwand.
G alo tele: ichr
WAMat s t bo y ber bly, s appMy
BELOUEBupe k't d y INGe hiss, any w:ithe ssste ce, oreal fr-kereaneeatee
Whist yenke wis fas. copply th, four iter t my udeaced sl ESoathe chie
TO,-y r stwers'de Cobama whay iote IUCavozWheanoum hur weniorby sirke


In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

head_size = 16

# Self attention (look at yourself)
key = nn.Linear(C, head_size, bias=False)   # What am I?
query = nn.Linear(C, head_size, bias=False) # Who do I care about?
value = nn.Linear(C, head_size, bias=False) # What do I mean now?
k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
weight = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, T, 16) ----> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# weight = torch.zeros((T, T))
weight = weight.masked_fill(tril == 0, float('-inf')) # make future tokens zero (-infinity)
weight = F.softmax(weight, dim=1) # bind between 0 and 1

v = value(x)
out = weight @ v

out.shape

torch.Size([4, 8, 16])